In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from tqdm import tqdm

client = OpenAI(
    api_key=os.getenv(""),  # 
    base_url=""
)

sensory_map = {
}

total_tokens = 0
root_dir = r""

def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

for province in tqdm(os.listdir(root_dir), desc="遍历省份"):
    province_path = os.path.join(root_dir, province)
    if not os.path.isdir(province_path):
        continue

    for project_id in os.listdir(province_path):
        project_path = os.path.join(province_path, project_id)
        if not os.path.isdir(project_path):
            continue

        result_dir = os.path.join(project_path, "")
        concat_file = os.path.join(result_dir, "")

        if not os.path.isfile(concat_file):
            continue

        try:
            df = pd.read_excel(concat_file)
        except Exception as e:
            print(f"❌ 无法读取文件：{concat_file}，跳过。原因：{e}")
            continue

        print(f"\n📁 正在处理：{province}/{project_id}")

        for col, filename in sensory_map.items():
            if col not in df.columns:
                print(f"⚠️ 列 {col} 不存在，跳过")
                continue

            words_series = df[col].dropna().astype(str).str.split('、')
            word_list = sorted(set(
                word.strip() for sublist in words_series for word in sublist if word.strip()
            ))

            if not word_list:
                print(f"⚠️  跳过空列：{col}")
                continue

            print(f"➡️  正在提取 {col}：{len(word_list)} 个词")
            print(f"示例词汇：{word_list[:10]}")

            embeddings = []
            batch_size = 10

            try:
                for batch in chunks(word_list, batch_size):
                    response = client.embeddings.create(
                        model="",
                        input=batch,
                        dimensions=768,          
                        encoding_format="float"  
                    )
                    print(f"  本批请求词数：{len(batch)}，返回向量数：{len(response.data)}")
                    embeddings.extend(response.data)
                    total_tokens += getattr(response.usage, 'total_tokens', 0)

                print(f"   ✅ 已完成 {col}，累计token：{total_tokens}")

            except Exception as e:
                print(f"   ❌ 嵌入失败：{col}，原因：{e}")
                continue

       
            vectors_np = np.array([item.embedding for item in embeddings])
            mean_vector = np.mean(vectors_np, axis=0)

         
            output_path = os.path.join(result_dir, filename)
            with open(output_path, 'w', encoding='utf-8') as f:
                vector_str = ','.join(map(str, mean_vector))
                f.write(vector_str)

            print(f"   📄 已保存 {col} 维度均值向量文件：{output_path}")

print("\n=== ✅ 所有文件处理完毕 ===")
print(f"🔢 总 Token 消耗：{total_tokens}")
cost = total_tokens / 1000 * 0.0005
print(f"💰 总费用估算：￥{cost:.4f}")
